# Importer une méthode d'Impact Assessment

In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import brightway2 as bw
import bw2io as bi
import bw2data as bd
import bw2analyzer as bw2
import bw2calc as bc
from brightway2 import *
from bw2data.parameters import ActivityParameter, DatabaseParameter, ProjectParameter, Group
from os.path import commonprefix
import tabulate

Il faut d'abord se placer dans le projet souhaité

In [2]:
bw.projects.set_current("my_project")

In [3]:
#Import méthode LCIA depuis un fichier Excel
lcia_method = bi.ExcelLCIAImporter(
    filepath=r"my_LCIA_file.xlsx", 
    name=("my_name", "my_precision_name"), 
    description="my_description" ,
    unit="my_unit",
    # On peut ajouter d'autres metadata
)

L'ExcelLCIAImporter a potentiellement considéré des lignes vides comme non-vides, il faut donc créer une méthode pour les éliminer:

In [4]:
def drop_empty_lines(data):
    for method in data:
        method['exchanges'] = [obj for obj in method['exchanges'] if obj['name']]
    return data

In [5]:
lcia_method.apply_strategy(drop_empty_lines)

Applying strategy: drop_empty_lines


Afin de mettre en forme les données issues de l'Excel dans le format exploitable par brightway, il faut faire:

In [8]:
lcia_method.apply_strategies()

Applying strategy: csv_restore_tuples
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: set_biosphere_type
Applying strategy: drop_unspecified_subcategories
Applying strategy: link_iterable_by_fields
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applied 8 strategies in 0.09 seconds


Pour vérifier que les liens avec la biosphère ont été correctement effectués, il faut qu'il y ait 0 unliked cfs en faisant:

In [9]:
lcia_method.statistics()

1 methods
6 cfs
0 unlinked cfs


(1, 6, 0)

Si tous les facteurs de caractérisations ne sont pas liés, on peut afficher les erreurs comme ceci:

In [11]:
for exc in lcia_method.unlinked:
    print(exc)

{'name': 'Carbon dioxide, from soil or biomass stock', 'categories': ('air', 'indoor'), 'amount': 3.53e-13, 'type': 'biosphere'}
{'name': 'Methane', 'categories': ('air', 'urban air close to ground'), 'amount': 1.59e-12, 'type': 'biosphere'}
{'name': 'NMVOC, non-methane volatile organic compounds, unspecified origin', 'categories': ('air',), 'amount': 1.06e-12, 'type': 'biosphere'}
{'name': 'NMVOC, non-methane volatile organic compounds, unspecified origin', 'categories': ('air', 'low population density, long-term'), 'amount': 1.06e-12, 'type': 'biosphere'}
{'name': 'NMVOC, non-methane volatile organic compounds, unspecified origin', 'categories': ('air', 'lower stratosphere + upper troposphere'), 'amount': 1.06e-12, 'type': 'biosphere'}
{'name': 'NMVOC, non-methane volatile organic compounds, unspecified origin', 'categories': ('air', 'non-urban air or from high stacks'), 'amount': 1.06e-12, 'type': 'biosphere'}
{'name': 'NMVOC, non-methane volatile organic compounds, unspecified orig

Sinon, on écrit la méthode dans le projet:

In [10]:
lcia_method.write_methods(overwrite=True)
#Si une méthode existe déjà avec le même nom et qu'on veut écrire par dessus (MàJ, etc..) il faut inscrire overwrite=True dans les parenthèses

Wrote 1 LCIA methods with 6 characterization factors
